In [1]:
include("./src/MPO_common.jl")
using .MPO_common
using JLD2
using ITensors
using LinearAlgebra
include("./src/iDMRG.jl")
using .iDMRG

Below we consider a 1d TFIM.

A note on inputting MPOs: Suppose we have a unit cell of size $n$, labeled $1,\cdots, n$. Each Hamiltonian term should be assigned the smallest index within this region, with the remaining indices specified by the appropriate increments, without taking indices modulo $n$.

In [2]:
uc = 4
jz = -1.0
hx = -2.0

oplist = MPOsum()
# ITensor defines spin-1/2 operators as sigma_j/2, therefore we compensate for the 1/2 factors in the coefficients.
for i = 1:uc
    mpoadd!(oplist, 4 * jz, (i, "Sz"), (i + 1, "Sz"))  # Note that the last term that goes out of this unit-cell has index (4,5), without modding out 4.
    mpoadd!(oplist, 2 * hx, (i, "Sx"))
end

sites = siteinds("S=1/2", uc)
iH = op_to_hm_inf(oplist, sites);  #Generate iMPO

# The output is a struct MPSdata, which contains the following data:
1. MPSdata.ψ contains the iMPS in the form of $A_1$, $B_2$, $B_3$,..., $B_n$, where $n$ is the number of sites in a unit-cell. A's are left canonical, and B's are right canonical
2. MPSdata.LP and MPSdata.RP are the environment, which can be useful if one wants to restart the iDMRG.
3. MPSdata.lambda_1, this is the Schmidt value $\Lambda_1$ between site 1 and site 2.
4. MPSdata.lambda_end, this is the Schmidt value $\Lambda_n$ between site $n$ and site 1.

Therefore the true periodic infinite MPS should be as follows (which then repeats itself indefinitely):

$[A_1\ \Lambda_1\ B_2\ B_3\ ...\ B_n\ \Lambda_n^{-1}]$

Cf., Binder Barthel, PRB 98,235114.

In [5]:
finalstate = idmrg_run(iH, sites; maxdimlist=[50, 100, 250], sweeplist=[2, 2, 3], cutoff=1e-12);

finish initialization
Maximum bond dimension is: 4
Energy is: -2.190482594288574
<Λ(n)|Λ(n+1)> overlap is: 0.9911349890866198
== Global sweep 1 ==
  Sweep 1
Maximum bond dimension is: 6
Energy is: -2.127088823323576
<Λ(n)|Λ(n+1)> overlap is: 0.9999999971823496
  Sweep 2
Maximum bond dimension is: 6
Energy is: -2.1270888199427027
<Λ(n)|Λ(n+1)> overlap is: 0.9999999999995569
== Global sweep 2 ==
  Sweep 1
Maximum bond dimension is: 6
Energy is: -2.1270888199426725
<Λ(n)|Λ(n+1)> overlap is: 0.9999999999995712
  Sweep 2
Maximum bond dimension is: 6
Energy is: -2.1270888199427063
<Λ(n)|Λ(n+1)> overlap is: 0.9999999999995705
== Global sweep 3 ==
  Sweep 1
Maximum bond dimension is: 6
Energy is: -2.1270888199426494
<Λ(n)|Λ(n+1)> overlap is: 0.9999999999995713
  Sweep 2
Maximum bond dimension is: 6
Energy is: -2.1270888199427134
<Λ(n)|Λ(n+1)> overlap is: 0.9999999999995716
  Sweep 3
Maximum bond dimension is: 6
Energy is: -2.127088819942742
<Λ(n)|Λ(n+1)> overlap is: 0.9999999999995722


The GS energy from iDMRG is $-2.127088819942742$, which can be compared with the exact energy given by $-1/2 \frac{1}{\pi}\int^{\pi}_{0}dk \epsilon_k$, where $\epsilon_k=2\sqrt(j^2+h^2-2jh\text{cos}(k))$.

In [9]:
using QuadGK
# dispersion
ε(k) = 2 * sqrt(jz^2 + hx^2 - 2jz*hx*cos(k))

# e.g. integrate over the 1D Brillouin zone [0, π]
gse = quadgk(k -> ε(k), 0, π; rtol=1e-10)[1]/2pi

println("Ground state energy = $gse")

Ground state energy = 2.12708881994673


# Transforming the states in canonical forms:
The above output is of course unwieldy. We can use the following two functions to transform it into canonical forms.

The idea is to transform an arbitrary iMPS $MMM$ into left-canonical or right-canonical form. We have $A=LML^{-1}$. Therefore $...MMMMM=...AAAAAL$, where one $L$ is pulled to left infinity. Similarly $B=R^{-1}MR$. Therefore $MMM...=RBBB...$.

From these we can also case the iMPS into a mixed canonical form which can then be used to compute correlation functions. The idea is, $...MMMM...=...AA(LR)BB...$, where $(LR)$ is the SVD of the center site. 

Need to normalize in the end. 

The algorithm we use is the simple iteration algorithm Eq.28 in Vanderstraeten et. al. Tangent-space methods for uniform MPS.

In [6]:
ψA,L1=left_canonical_form(finalstate,sites); # I used an iteration algorithm so the output is a convergence criteria which is not material.

Converged after 1 sweeps with error 9.430443537183919e-9
Below is the result
1.0000000036944388 + 0.0im


In [7]:
ψB,Rl=right_canonical_form(finalstate,sites); 

Converged after 6 sweeps with error 3.40152664724197e-9
Below is the result
0.99999999999091 + 3.3419001543490367e-14im


In [8]:
Λc = L1 * Rl
Λc = Λc / sqrt(tr(array(Λc) * transpose(conj(array(Λc)))))

ITensor ord=2 (dim=6|id=175|"Link,qr") (dim=6|id=696|"Link,rq")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

The final iMPS in the mixed gauge is therefore $\cdots \psi_A\ \psi_A\ \Lambda_c\ \psi_B\ \psi_B\ \cdots$